In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!ls ../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP/ChCh-Miner_durgbank-chem-chem.tsv

../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP/ChCh-Miner_durgbank-chem-chem.tsv


In [3]:
ddi = pd.read_csv('../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP/ChCh-Miner_durgbank-chem-chem.tsv', sep = '\t', header = None)

In [4]:
len(set(ddi[0].tolist() + ddi[1].tolist()))

1514

In [5]:
pd.DataFrame(list(set(ddi[0].tolist() + ddi[1].tolist()))).rename(columns={0: "Drug1_ID"}).to_csv('../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP_Full/ddi_unique_smiles.csv')

In [6]:
drug_list = pd.DataFrame(list(set(ddi[0].tolist() + ddi[1].tolist()))).rename(columns={0: "Drug1_ID"})

In [7]:
len(ddi)

48514

In [8]:
drug_list.values.T[0]

array(['DB00424', 'DB00033', 'DB01222', ..., 'DB01007', 'DB00264',
       'DB08872'], dtype=object)

In [9]:
from itertools import combinations 
comb = combinations(list(drug_list.values.T[0]), 2)

In [10]:
comb = list(comb)

In [11]:
import random

In [12]:
pos = [(i[0], i[1]) for i in (ddi.values)]

In [13]:
neg = list(set(comb) - set(pos))

In [14]:
comb_flipped = [(i[1], i[0]) for i in comb]

In [15]:
neg_2 = list(set(comb_flipped) - set(pos))

In [16]:
neg_2 = [(i[1], i[0]) for i in neg_2]

In [17]:
len(neg)

1119471

In [18]:
neg_final = list(set(neg) & set(neg_2))

In [19]:
len(neg_final)

1096827

In [20]:
len(set(comb))

1145341

In [21]:
random.seed(a = 1)
neg_sample = random.sample(neg_final, len(ddi))

In [22]:
len(neg_sample)

48514

In [23]:
df = pd.DataFrame(pos+neg_sample)
df['label'] = np.array([1]*len(pos) + [0]*len(neg_sample))

In [24]:
df = df.rename({0:'Drug1_ID', 1:'Drug2_ID'}, axis = 1)

In [105]:
def create_fold(df, x):
    test = df.sample(frac = 0.2, replace = False, random_state = x)
    train_val = df[~df.index.isin(test.index)]
    val = train_val.sample(frac = 0.125, replace = False, random_state = 1)
    train = train_val[~train_val.index.isin(val.index)]
    
    path = '../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP_Full/fold'+str(x)
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')
    test.reset_index(drop = True).to_csv(path + '/test.csv')
    train_val.reset_index(drop = True).to_csv(path + '/train_val.csv')
    
    return train, val, test

In [104]:
!mkdir ../../../../scratch/kh2383/IGCN/data/DDI/BIOSNAP_Full/fold5

In [110]:
train, val, test = create_fold(df, 5)